# Taxi Company Investigation

In this Jupyter notebook, we investigate some synthetic data to understand the value of probability distributions and how they can be used to make predictions about the possible values of future data points. However, we also see why care needs to be taken to ensure that data is being looked at in the right way.

This notebook uses lots of Python code that you will not be familiar with, but you don't need to understand exactly how the code works. Just read through the data story, run the cells, look at the outputs and consider what you are learning about the data as you go.

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('feb_taxi_calls.csv')

The data contains information about the number of calls to a taxi company for twenty minute intervals for every week day in February. The time is measured in minutes past midnight.

We are going to assume it takes just 20 minutes to attend to a call (pretty impressive service) and that each call is attended to within the corresponding time interval. Clearly an oversimplification, but could be developed into a more realistic model.

In [ ]:
data[0:10]

Here are some summary statistics:

In [ ]:
print("Number of data points =", len(data))
print("Mean     =", data['calls'].mean())
print("St. Dev. =", data['calls'].std())

Since the data relates to the number of events occurring in an interval, we would expect it to be Poisson distributed. The Poisson distribution has one parameter (the mean rate of events per interval), which we estimate with the mean of the sample:

In [ ]:
mu = data['calls'].mean()

Say that we want to ensure that we have enough cars to cover all the calls at least 99% of the time.

By calculating confidence intervals of the Poisson distribution, we can find out how many cars we need:

In [ ]:
pc99 = sps.poisson.ppf(0.99,mu)
pc98int = sps.poisson.interval(0.98,mu)
pc98list = list(range(int(pc98int[0]),int(pc98int[1]+1)))

print("At least 99% of time intervals will have", int(pc99), "calls or fewer.")
print("Alternatively, at least 98% of time intervals will have a number of calls in the interval", str(pc98list) + ".")

In [ ]:
print("Based on our assumptions,", int(pc99), "cars should be enough in at least 99% of cases.")

## Are there any reasons to doubt this conclusion?

If this conclusion is right, there should only be about 14 time intervals (or fewer) with greater than 6 calls (1% of 1440).

In [ ]:
busy_intervals = data[data['calls'] > 6]
busy_intervals

In [ ]:
print("There are", len(busy_intervals), "intervals with more than 6 calls.")

Something seems to have gone wrong. Perhaps the data is not Poisson distributed.

We failed to visualise the data. Let's do that now:

In [ ]:
n,bins,patches = plt.hist(data['calls'], 20, range=(0,20))

This certainly looks like a Poisson distribution (see https://www.umass.edu/wsp/images/poisson3.gif).

Let's plot the individual call numbers for each interval, to see if there is some hidden pattern that could be interfering with the results:

In [ ]:
plt.plot(data['calls'],'.')

There is no visible pattern.

These graphs do not show anything suspicious to explain the higher than expected number of time intervals with high call volumes (>6 calls).

...

But if we think about what the data actually represents, the problem should be obvious.

Let's group the data by the time of day, and plot the average number of calls for each time:

In [ ]:
grouped = data.groupby('time')
mean_by_time = grouped.agg('mean')
plt.plot(mean_by_time)

Clearly, as we should have realised, the number of calls varies significantly by time of day. We need a different Poisson distribution for each time of day.

Let's calculate new expected maximum call numbers for each time of day:

In [ ]:
max_exp_customers = sps.poisson.ppf(0.99,mean_by_time['calls'])
max_exp_customers = np.ravel(max_exp_customers) # This line just turns the array into a row, rather than a column.
mean_by_time['99th Percentile'] = max_exp_customers

In [ ]:
time_1 = [120, "02:00"]
time_2 = [1140,"19:00"]

mean_1 = mean_by_time['calls'][time_1[0]]
mean_2 = mean_by_time['calls'][time_2[0]]

max_1  = int(mean_by_time['99th Percentile'][time_1[0]])
max_2  = int(mean_by_time['99th Percentile'][time_2[0]])

print("For example, at", time_1[1], "the mean number of calls is", mean_1, "and 99% of days will have", max_1, "calls or fewer at this time.")
print("In contrast, at", time_2[1], "the mean number of calls is", mean_2, "and 99% of days will have", max_2, "calls or fewer at this time.")


A sensible conclusion might be that there should be more cars available at different times of day, as we might expect.